# 드라이브 연동

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [75]:
!pip freeze >> requirements.txt

# 라이브러리 설치 및 선언

In [8]:
!apt install ffmpeg -y
!pip install pydub

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [10]:
import os
import csv
import librosa
import pandas as pd
import numpy as np


from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import load_model
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense


import matplotlib.pyplot as plt
from pydub import AudioSegment

import joblib
mel_scaler = joblib.load('/content/drive/MyDrive/과정운영/KSEB/2025년도/프로젝트 결과/04. 김요한_전호연_김시명팀/김요한/mel_scaler.pkl')

# 예측

In [11]:
import os
import csv
import librosa
import pandas as pd
import numpy as np


from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import load_model
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense


import matplotlib.pyplot as plt
from pydub import AudioSegment


## 오디오 업로드 및 mel-spectrogram 작업

In [12]:
###################LOUDNESS###################

model_loudness = load_model("/content/drive/MyDrive/과정운영/KSEB/2025년도/프로젝트 결과/04. 김요한_전호연_김시명팀/김요한/loudness_model.keras")
loudness_scaler = MinMaxScaler() # 추론 시점에 새로 다시 선언
loudness_scaler.data_min_ = np.load("/content/drive/MyDrive/과정운영/KSEB/2025년도/프로젝트 결과/04. 김요한_전호연_김시명팀/김요한/loudness_scaler_min_.npy")
loudness_scaler.scale_ = np.load("/content/drive/MyDrive/과정운영/KSEB/2025년도/프로젝트 결과/04. 김요한_전호연_김시명팀/김요한/loudness_scaler_scale_.npy")
loudness_scaler.min_ = 0
loudness_scaler.data_max_ = loudness_scaler.data_min_ + 1 / loudness_scaler.scale_



###################VALENCE###################


model_valence = load_model("/content/drive/MyDrive/과정운영/KSEB/2025년도/프로젝트 결과/04. 김요한_전호연_김시명팀/김요한/valence_model.keras")
valence_scaler = MinMaxScaler() # 추론 시점에 새로 다시 선언
valence_scaler.data_min_ = np.load("/content/drive/MyDrive/과정운영/KSEB/2025년도/프로젝트 결과/04. 김요한_전호연_김시명팀/김요한/valence_scaler_min_.npy")
valence_scaler.scale_ = np.load("/content/drive/MyDrive/과정운영/KSEB/2025년도/프로젝트 결과/04. 김요한_전호연_김시명팀/김요한/valence_scaler_scale_.npy")
valence_scaler.min_ = 0
valence_scaler.data_max_ = valence_scaler.data_min_ + 1 / valence_scaler.scale_



###################TEMPO###################

model_tempo = load_model("/content/drive/MyDrive/과정운영/KSEB/2025년도/프로젝트 결과/04. 김요한_전호연_김시명팀/김요한/tempo_model.keras")
tempo_scaler = MinMaxScaler() # 추론 시점에 새로 다시 선언
tempo_scaler.data_min_ = np.load("/content/drive/MyDrive/과정운영/KSEB/2025년도/프로젝트 결과/04. 김요한_전호연_김시명팀/김요한/tempo_scaler_min_.npy")
tempo_scaler.scale_ = np.load("/content/drive/MyDrive/과정운영/KSEB/2025년도/프로젝트 결과/04. 김요한_전호연_김시명팀/김요한/tempo_scaler_scale_.npy")
tempo_scaler.min_ = 0
tempo_scaler.data_max_ = tempo_scaler.data_min_ + 1 / tempo_scaler.scale_

## 모델 및 scaler 불러오기

In [20]:
from google.colab import files

uploaded = files.upload()
filename = list(uploaded.keys())[0]

y, sr = librosa.load(filename, sr=16000)
mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
mel_db = librosa.power_to_db(mel, ref=np.max)

## 위에서 사용한 mel_scaler 재사용함
mel_db = mel_scaler.transform(mel_db.reshape(-1, 1)).reshape(mel_db.shape)
mel_db = np.expand_dims(mel_db, axis=-1) # (128, T) → (128, T, 1)


target_length = 1876 # 학습 시 사용한 길이와 동일
if mel_db.shape[1] < target_length:
  mel_db = np.pad(mel_db, ((0, 0), (0, target_length - mel_db.shape[1]), (0, 0)), mode='constant')
else:
  mel_db = mel_db[:, :target_length, :]

mel_db = np.expand_dims(mel_db, axis=0) # (1, 128, 1876, 1)

Saving 3 REFLECTIONS OF YOU.MP3 to 3 REFLECTIONS OF YOU.MP3


## 예측

In [21]:
# loudness 예측
pred_loudness_scaled = model_loudness.predict(mel_db)
pred_loudness = loudness_scaler.inverse_transform(pred_loudness_scaled)
# print(f"{pred_loudness[0][0]:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step


In [22]:
# valence 예측
pred_valence_scaled = model_valence.predict(mel_db)
pred_valence = valence_scaler.inverse_transform(pred_valence_scaled)
# print(f"{pred_valence[0][0]:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step


In [23]:
# tempo 예측
pred_tempo_scaled = model_tempo.predict(mel_db)
pred_tempo = tempo_scaler.inverse_transform(pred_tempo_scaled)
# print(f"{pred_tempo[0][0]:.2f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step


In [63]:
deep_learning_result = {
    'loudness': float(f"{pred_loudness[0][0]:.2f}"),
    'valence': float(f"{pred_valence[0][0]:.2f}"),
    'tempo': float(f"{pred_tempo[0][0]:.2f}"),
}
deep_learning_result

{'loudness': 34.88, 'valence': 0.73, 'tempo': 96.41}

### 시각화

### 라이브러리

In [25]:
# 데이터를 처리 라이브러리
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 머신러닝 라이브러리 데이터 분리 70% 30%
from sklearn.model_selection import train_test_split
# 모델 관련 라이브러리
from sklearn.tree import DecisionTreeRegressor

### 데이터 불러오기

In [26]:
featuresData = pd.read_csv("/content/drive/MyDrive/과정운영/KSEB/2025년도/프로젝트 결과/04. 김요한_전호연_김시명팀/csv/universal_top_spotify_songs.csv")
featuresData.head(2)

,spotify_id,name,artists,daily_rank,daily_movement,weekly_movement,country,snapshot_date,popularity,is_explicit,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,42UBPzRMh5yyz0EDPr6fr1,Manchild,Sabrina Carpenter,1,3,49,NaN,2025-06-10,87,True,...,7,-5.087,1,0.0572,0.122,0.000000,0.317,0.811,123.010,4
1,2RkZ5LkEzeHGRsmDqKwmaJ,Ordinary,Alex Warren,2,-1,-1,NaN,2025-06-10,95,False,...,2,-6.141,1,0.0600,0.704,0.000007,0.055,0.391,168.115,3


In [27]:
featuresData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2106716 entries, 0 to 2106715
Data columns (total 25 columns):
 #   Column              Dtype  
---  ------              -----  
 0   spotify_id          object 
 1   name                object 
 2   artists             object 
 3   daily_rank          int64  
 4   daily_movement      int64  
 5   weekly_movement     int64  
 6   country             object 
 7   snapshot_date       object 
 8   popularity          int64  
 9   is_explicit         bool   
 10  duration_ms         int64  
 11  album_name          object 
 12  album_release_date  object 
 13  danceability        float64
 14  energy              float64
 15  key                 int64  
 16  loudness            float64
 17  mode                int64  
 18  speechiness         float64
 19  acousticness        float64
 20  instrumentalness    float64
 21  liveness            float64
 22  valence             float64
 23  tempo               float64
 24  time_signature      int6

In [28]:
ynDict = {}
ynDict ={np.True_: 1, np.False_:0}
featuresData["is_explicit_LE"] = featuresData.is_explicit.map(ynDict)

In [29]:
featuresData

,spotify_id,name,artists,daily_rank,daily_movement,weekly_movement,country,snapshot_date,popularity,is_explicit,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,is_explicit_LE
0,42UBPzRMh5yyz0EDPr6fr1,Manchild,Sabrina Carpenter,1,3,49,NaN,2025-06-10,87,True,...,-5.087,1,0.0572,0.122000,0.000000,0.3170,0.811,123.010,4,1
1,2RkZ5LkEzeHGRsmDqKwmaJ,Ordinary,Alex Warren,2,-1,-1,NaN,2025-06-10,95,False,...,-6.141,1,0.0600,0.704000,0.000007,0.0550,0.391,168.115,3,0
2,0FTmksd2dxiE5e3rWyJXs6,back to friends,sombr,3,-1,0,NaN,2025-06-10,98,False,...,-2.291,1,0.0301,0.000094,0.000088,0.0929,0.235,92.855,4,0
3,7so0lgd0zP2Sbgs2d7a1SZ,Die With A Smile,"Lady Gaga, Bruno Mars",4,1,0,NaN,2025-06-10,91,False,...,-7.727,0,0.0317,0.289000,0.000000,0.1260,0.498,157.964,3,0
4,27xkOIER6uDLKALIelHylZ,Don’t Say You Love Me,Jin,5,-2,-3,NaN,2025-06-10,92,False,...,-5.490,1,0.0304,0.484000,0.000000,0.1050,0.757,111.976,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106711,0AYt6NMyyLd0rLuvr0UkMH,Slime You Out (feat. SZA),"Drake, SZA",46,4,0,AE,2023-10-18,84,True,...,-9.243,0,0.0502,0.508000,0.000000,0.2590,0.105,88.880,3,1
2106712,2Gk6fi0dqt91NKvlzGsmm7,SAY MY GRACE (feat. Travis Scott),"Offset, Travis Scott",47,3,0,AE,2023-10-18,80,True,...,-5.060,1,0.0452,0.058500,0.000000,0.1320,0.476,121.879,4,1
2106713,26b3oVLrRUaaybJulow9kz,People,Libianca,48,2,0,AE,2023-10-18,88,False,...,-7.621,0,0.0678,0.551000,0.000013,0.1020,0.693,124.357,5,0
2106714,5ydjxBSUIDn26MFzU3asP4,Rainy Days,V,49,1,0,AE,2023-10-18,88,False,...,-8.016,0,0.0875,0.739000,0.000000,0.1480,0.282,74.828,4,0


In [30]:
RelDateList = list( featuresData.album_release_date.drop_duplicates().values)
RelDateList = pd.DataFrame(RelDateList)

In [32]:
def get_season(date_str):
    if pd.isna(date_str):
        return None
    try:
        # 날짜를 문자열로 변환
        date_obj = datetime.strptime(str(date_str), '%Y-%m-%d')
        month = date_obj.month
        if 3 <= month <= 5:
            return '봄'
        elif 6 <= month <= 8:
            return '여름'
        elif 9 <= month <= 11:
            return '가을'
        else:
            return '겨울'
    except ValueError:
        return None

In [33]:
featuresData['season'] = featuresData['album_release_date'].apply(get_season)

In [34]:
featuresData.head(5)

,spotify_id,name,artists,daily_rank,daily_movement,weekly_movement,country,snapshot_date,popularity,is_explicit,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,is_explicit_LE,season
0,42UBPzRMh5yyz0EDPr6fr1,Manchild,Sabrina Carpenter,1,3,49,NaN,2025-06-10,87,True,...,1,0.0572,0.122000,0.000000,0.3170,0.811,123.010,4,1,여름
1,2RkZ5LkEzeHGRsmDqKwmaJ,Ordinary,Alex Warren,2,-1,-1,NaN,2025-06-10,95,False,...,1,0.0600,0.704000,0.000007,0.0550,0.391,168.115,3,0,가을
2,0FTmksd2dxiE5e3rWyJXs6,back to friends,sombr,3,-1,0,NaN,2025-06-10,98,False,...,1,0.0301,0.000094,0.000088,0.0929,0.235,92.855,4,0,겨울
3,7so0lgd0zP2Sbgs2d7a1SZ,Die With A Smile,"Lady Gaga, Bruno Mars",4,1,0,NaN,2025-06-10,91,False,...,0,0.0317,0.289000,0.000000,0.1260,0.498,157.964,3,0,봄
4,27xkOIER6uDLKALIelHylZ,Don’t Say You Love Me,Jin,5,-2,-3,NaN,2025-06-10,92,False,...,1,0.0304,0.484000,0.000000,0.1050,0.757,111.976,4,0,봄


In [35]:
import numpy as np

In [36]:
countryList = list( featuresData.country.drop_duplicates().values)

In [37]:
i=2

In [38]:
fec1 = featuresData.loc[ featuresData.country == countryList[i] ]

In [39]:
fec1.corr(numeric_only=True)

,daily_rank,daily_movement,weekly_movement,popularity,is_explicit,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,is_explicit_LE
daily_rank,1.000000,-0.146866,-0.266543,-0.344916,-0.114011,0.144735,-0.013355,-0.228431,0.034291,-0.207272,0.006972,-0.045596,0.230221,0.090889,-0.019877,-0.168842,0.092186,-0.011311,-0.114011
daily_movement,-0.146866,1.000000,0.340871,-0.125452,0.005222,-0.001155,-0.008199,0.043023,-0.011413,0.042657,-0.006819,0.009383,-0.037943,-0.007506,-0.001189,0.007715,0.002795,0.010058,0.005222
weekly_movement,-0.266543,0.340871,1.000000,-0.128652,0.008049,-0.003265,-0.012219,0.065203,-0.013037,0.063120,-0.008051,0.017050,-0.059798,-0.013712,-0.012998,0.012359,-0.010305,0.017938,0.008049
popularity,-0.344916,-0.125452,-0.128652,1.000000,0.162156,-0.256886,-0.037169,0.269934,-0.021949,0.293708,-0.097057,-0.029648,-0.329675,-0.069029,0.057317,0.214965,-0.122551,-0.036307,0.162156
is_explicit,-0.114011,0.005222,0.008049,0.162156,1.000000,-0.292650,0.203341,0.217241,0.127498,0.083806,-0.077619,0.101314,-0.180502,-0.058086,-0.040406,0.182341,-0.005019,-0.044092,1.000000
duration_ms,0.144735,-0.001155,-0.003265,-0.256886,-0.292650,1.000000,-0.195719,-0.161606,-0.114246,-0.094947,0.218679,-0.196060,0.267541,-0.017893,0.054931,-0.247963,0.074906,-0.013507,-0.292650
danceability,-0.013355,-0.008199,-0.012219,-0.037169,0.203341,-0.195719,1.000000,0.152328,0.153595,0.009217,-0.156591,0.101862,-0.190073,-0.049055,0.090398,0.512983,-0.136272,0.278952,0.203341
energy,-0.228431,0.043023,0.065203,0.269934,0.217241,-0.161606,0.152328,1.000000,-0.062428,0.757965,-0.169147,0.123556,-0.585694,-0.162346,0.062493,0.427962,0.046324,0.209482,0.217241
key,0.034291,-0.011413,-0.013037,-0.021949,0.127498,-0.114246,0.153595,-0.062428,1.000000,-0.120727,-0.079645,-0.085297,-0.010328,0.045990,-0.044384,0.029967,0.114119,0.056765,0.127498
loudness,-0.207272,0.042657,0.063120,0.293708,0.083806,-0.094947,0.009217,0.757965,-0.120727,1.000000,-0.086748,-0.060348,-0.395438,-0.216349,0.012094,0.320597,0.071937,0.046701,0.083806


In [40]:
featuresData.columns[featuresData.isnull().any()]

Index(['name', 'artists', 'country', 'album_name', 'album_release_date',
       'season'],
      dtype='object')

In [41]:
featuresData.shape[0]

2106716

In [42]:
nullCheckDf=featuresData.isnull().sum()

In [43]:
nullCheckDf = pd.DataFrame(nullCheckDf)

In [44]:
nullCheckDf.columns = ["null_cnt"]

In [45]:
nullCheckDf["total_cnt"]=featuresData.shape[0]

In [46]:
nullCheckDf["null_ratio"] = np.where ( nullCheckDf.total_cnt == 0, 0, nullCheckDf.null_cnt / nullCheckDf.total_cnt)

In [47]:
nullCheckDf

,null_cnt,total_cnt,null_ratio
spotify_id,0,2106716,0.000000
name,30,2106716,0.000014
artists,29,2106716,0.000014
daily_rank,0,2106716,0.000000
daily_movement,0,2106716,0.000000
weekly_movement,0,2106716,0.000000
country,28858,2106716,0.013698
snapshot_date,0,2106716,0.000000
popularity,0,2106716,0.000000
is_explicit,0,2106716,0.000000


In [48]:
# 'country' 또는 'album_release_date' 컬럼에 결측치가 있는 행 제거
featuresData_cleaned = featuresData.dropna(subset=['country', 'album_release_date']).copy()

# 결측치 제거 후 해당 컬럼의 결측치 확인
print("\n결측치 제거 후 'country' 컬럼의 결측치 개수:", featuresData_cleaned['country'].isnull().sum())
print("결측치 제거 후 'album_release_date' 컬럼의 결측치 개수:", featuresData_cleaned['album_release_date'].isnull().sum())


결측치 제거 후 'country' 컬럼의 결측치 개수: 0
결측치 제거 후 'album_release_date' 컬럼의 결측치 개수: 0


In [49]:
featuresData.columns

Index(['spotify_id', 'name', 'artists', 'daily_rank', 'daily_movement',
       'weekly_movement', 'country', 'snapshot_date', 'popularity',
       'is_explicit', 'duration_ms', 'album_name', 'album_release_date',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'time_signature', 'is_explicit_LE', 'season'],
      dtype='object')

### 특성선정

In [50]:
# 국가, 계절 별 인기도 구하기
groupKey = ["country","season"]

In [51]:
modelDict={}

In [52]:
groupData = featuresData.groupby( groupKey )

In [53]:
eachGroup = groupData.get_group( list( groupData.groups )[0] )

In [54]:
corrDfList = list( range(3,8,1) )

In [55]:
list(range(8,3, 1))

[]

In [56]:
corrDfList = list( np.array( corrDfList ) /10 )

In [57]:
def groupbyMl(eachGroup):
    eachGroup = eachGroup.reset_index(drop=True)
    eachCountryID = eachGroup.loc[0, groupKey[0]]
    eachSeasonID = eachGroup.loc[0, groupKey[1]]

    label = ["popularity"]
    corrDf = eachGroup.corr(numeric_only=True)

    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestRegressor

    testData = pd.DataFrame()  # 마지막 모델 기준 예측 결과용 (원한다면 바꿀 수 있음)

    for corrStd in sorted(corrDfList, reverse=True):
        features = list(corrDf.loc[
            (abs(corrDf["popularity"]) > corrStd) &
            (corrDf["popularity"] != 1)
        ].index)

        if not features:
            continue  # features 없으면 건너뜀

        print(f"✅ 모델 저장 대상: {eachCountryID}-{eachSeasonID}, corrStd: {corrStd}, features: {features}")

        # 모델 훈련
        trainData, testDataLocal = train_test_split(eachGroup, random_state=10, test_size=0.2)
        trainingDataFeatures = trainData[features]
        trainingDataLabel = trainData[label]
        testDataFeatures = testDataLocal[features]

        modelRf = RandomForestRegressor(n_estimators=50, random_state=10)
        fittedRf = modelRf.fit(trainingDataFeatures, trainingDataLabel)

        # ✅ modelDict에 corrStd 기준으로 모델 저장
        modelDict[(eachCountryID, eachSeasonID, corrStd)] = [fittedRf, features]

        # 마지막 모델 기준 예측 결과 저장 (원한다면 다수 저장 방식도 가능)
        testData = testDataLocal.copy()
        testData["PREDICT_RF"] = fittedRf.predict(testDataFeatures)

    if testData.empty:
        print(f"❌ No features selected for any corrStd in group {eachCountryID}-{eachSeasonID}.")
    return testData


In [ ]:
mergedMlDf = featuresData.groupby( groupKey ).apply(groupbyMl)

In [59]:
mergedMlDf

spotify_id  \
country season                                
AE      겨울     1832  3wPPWcVuinAU7dXcJXtCID   
               4448  0TL0LFcwIBF5eX7arDIKxY   
               1363  4u43I0LP2Xf85OAS85eG0R   
               4625  1IcR6RlgvDczfvoWJSSY2A   
               3894  1zzejMGRYKP5XOa3FmzXfa   
...                                     ...   
ZA      여름     4120  09zHC0CNGuWQ1ot7WhzwKJ   
               2557  2hrycoFU1mZw6YPvMcn8yC   
               5260  5Jf3hmkf8WKKqxM4vM8rux   
               2450  21fhgH4tjYcAKzkzCuYyr6   
               3130  22DH8NChecsgPxDjA4pqer   

                                                                  name  \
country season                                                           
AE      겨울     1832                                           La La La   
               4448                                               Husn   
               1363                                           CN TOWER   
               4625           It's the Most Wonderful Time of the Year   
               3894                   Katchi Sera - From "Think Indie"   
...                                                                ...   
ZA      여름     4120                                     Uyangijabulisa   
               2557                                            KEHLANI   
               5260                                              Eyami   
               2450  Sdakiwe Sbali (feat. Bless The Gentleman & DJ ...   
               3130                                              Tough   

                                                     artists  daily_rank  \
country season                                                             
AE      겨울     1832                   Naughty Boy, Sam Smith        38.0   
               4448                                Anuv Jain        25.0   
               1363                     PARTYNEXTDOOR, Drake        29.0   
               4625                            Andy Williams        47.0   
               3894                           Sai Abhyankkar        20.0   
...                                                      ...         ...   
ZA      여름     4120     Fezeka Dlamini, Nomfundo Moh, Naledi        33.0   
               2557                          Jordan Adetunji        13.0   
               5260                           Lwah Ndlunkulu        17.0   
               2450  Blaq Major, BLESS THE GENTLEMAN, DJ KAP        14.0   
               3130                      Quavo, Lana Del Rey        40.0   

                     daily_movement  weekly_movement country snapshot_date  \
country season                                                               
AE      겨울     1832            -6.0             12.0      AE    2025-01-20   
               4448             3.0             25.0      AE    2023-12-30   
               1363             8.0            -21.0      AE    2025-02-22   
               4625           -11.0              0.0      AE    2023-12-13   
               3894            -2.0             15.0      AE    2024-02-18   
...                             ...              ...     ...           ...   
ZA      여름     4120             2.0              2.0      ZA    2024-01-08   
               2557            -6.0              2.0      ZA    2024-08-17   
               5260            33.0              0.0      ZA    2023-10-18   
               2450             0.0             -1.0      ZA    2024-08-23   
               3130             2.0             10.0      ZA    2024-07-12   

                     popularity is_explicit  ...  speechiness acousticness  \
country season                               ...                             
AE      겨울     1832        80.0       False  ...       0.0306      0.10700   
               4448        79.0       False  ...       0.0298      0.68200   
               1363        78.0        True  ...       0.1940      0.31800   
               4625        78.0       False  ...       0.0411   

In [60]:
modelDict

{('AE',
  '겨울',
  np.float64(0.3)): [RandomForestRegressor(n_estimators=50, random_state=10), ['daily_movement',
   'weekly_movement']],
 ('AE',
  '봄',
  np.float64(0.3)): [RandomForestRegressor(n_estimators=50, random_state=10), ['daily_movement',
   'weekly_movement']],
 ('AR',
  '가을',
  np.float64(0.3)): [RandomForestRegressor(n_estimators=50, random_state=10), ['speechiness']],
 ('AR',
  '겨울',
  np.float64(0.4)): [RandomForestRegressor(n_estimators=50, random_state=10), ['loudness',
   'instrumentalness',
   'valence']],
 ('AR',
  '겨울',
  np.float64(0.3)): [RandomForestRegressor(n_estimators=50, random_state=10), ['daily_rank',
   'is_explicit',
   'energy',
   'loudness',
   'acousticness',
   'instrumentalness',
   'valence',
   'is_explicit_LE']],
 ('AR',
  '봄',
  np.float64(0.3)): [RandomForestRegressor(n_estimators=50, random_state=10), ['daily_movement',
   'weekly_movement']],
 ('AT',
  '겨울',
  np.float64(0.3)): [RandomForestRegressor(n_estimators=50, random_state=10), ['wee

In [61]:
loadedModel = modelDict[('PY', '가을', np.float64(0.4))]

In [62]:
loadedRfModel = loadedModel[0]

In [64]:
inLoudness = deep_learning_result['loudness']
inValence = deep_learning_result['valence']
inTempo = deep_learning_result['tempo']

In [65]:
futureDf = pd.DataFrame( [[inLoudness,inValence,inTempo]] )

In [68]:
PredictDt = loadedRfModel.predict(futureDf)
PredictDt

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


array([55.68329441])

In [73]:
f'Spotify의 경향에 따르면, 사용자가 업로드한 {filename}의 인기도는 {round(PredictDt[0], 2)}로 추정됩니다.'

'Spotify의 경향에 따르면, 사용자가 업로드한 3 REFLECTIONS OF YOU.MP3의 인기도는 55.68로 추정됩니다.'